# 支持向量机
支持向量机是强有力的使用广泛的机器学习模型，可以支持线性和非线性的分类，回归，甚至离群值检测。  
支持向量机特别适合复杂的小型或中性数据集。  
## 线性SVM分类
支持向量机的核心思想是使用一个超平面将类别分割开，并且使最近的两个样例点与平面距离达到最大。  
SVM分类将类别用一条尽可能宽的街道分隔开，这叫做大间隔分类方法。  
不在分隔道的样例不会对svm产生任何影响，它只取决于在分隔道边的样例，这些样例被称为支持向量  
支持向量机方法还对样例数值范围敏感，所以使用前需要scale。  
## 软间隔分类  
严格要求所有样例都在分隔道外的方法称为硬间隔hard margin classification，这种方法对于离群值很敏感。  
为了避免因为离群值导致无法间隔分类，我们限制分隔违反margin violations，这种方法就是soft margin classification。  
在sklearn中使用超参数C来描述分隔违反，C越小，分隔道越宽，分隔违反越严重。  
如果SVM过拟合，可以正则化C来抑制。

In [14]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline((
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss='hinge')),
        ))

svm_clf.fit(X, y)

svm_clf.predict([[5.5, 1.7]])

array([ 1.])

与其他分类方法不同，svm不能输出样例对于每个类的概率。  
也可以使用SVC(kenerl="linear",C=1)来实现，不过这种方法很慢，特别是面对大量数据的情况，  
另一个方法就是用SGDClassifier(loss="hinge",alpha=1/(m*C)，这种方法应用SGD去训练线性SVM模型，他不一定比linearsvc快，但是在处理海量数据，或者应用在线学习时非常有用。  


# 非线性SVM分类
## polyfeature方法
有些数据集不是线性可分的，而使用多次项作为新特征又可分，类似多项式线性分类：  
sklearn中我们可以在pipeline中加入polyfeature来实现这种分类。
## polynoiaml kernal方法
加入多项式特征可以很好的应用在机器学习的各种算法上，但是低degree不能处理复杂数据，过高degree又导致新建特征太多，很容易模型过慢。 
幸运的svm有核方法kernal trick。我们可以做出和多项式特征方法一样的结果。coef0参数控制高次项对低次项的影响比例。
## 加入相似特征
另一个处理非线性问题的办法是使用相似的函数加入新特征，从而测量每个样例相似的位置。
下限情况是m个样例加入m的特征来区分  
高斯rbf核  
$\phi\gamma(x,\ell)=exp(-\gamma||x-\ell||^2)$  
## 如何选择合适的核
首先尝试linear线性核，特别是训练集比较大或者有足够的特征时；  
当训练集比较小时，可以尝试RBF核，通常很有用，  
如果你还有足够的时间，再去尝试其他核，使用cv和gridsearch选择合适的参数。
# SVM回归
和SVC不同，SVR是尽可能的使样例出现在分隔道上，道宽为超参数$\epsilon$,预测曲线就是分隔道中心线。

# SVM的机制
## 决策函数和预测
SVM中我们将各项特征的系数向量称为$\mathbf w$，截距称作$b$,则有方程如下:      
$\hat{y}=\left{\begin{array}{ll}0&if\quad\mathbf w^T\cdot x+b<0,\\1&if\quad\mathbf w^T\cdot x+b\ge0\end{array}\right.$    
  
  其中$\mathbf w^T\cdot x+b是决策函数$，实际上分隔道中心线就是决策函数与特征空间重叠的部分。  
## 训练目标
越大的斜率$||\mathbf w||$导致越小的间隔。如果要避免间隔违反即实现hard margin，我们要使决策函数在所有正例输入下，输出大于1，反例输入下，输出小于-1。如果我们定义  
$t^{(i)}=\left\{\begin{array}{ll} 1&if\quad y^{(i)}=1\\-1&if\quad y^{(i)}=0\end{array}\right.$  
那么有硬分隔目标如下：  
$minimize_{\mathbf w,b}\qquad \frac{1}{2}\mathbf w^T\cdot\mathbf w$  
$subject\quad to\quad t^{(i)}(\mathbf w^T\cdot x^{(i)}+b)\ge 1 \quad for\quad i=1,2\cdots ,m$  
  
为了实现软分隔，我们加入从变量$\zeta^{(i)}\ge 0$用来表示第i个样例允许的间隔违反程度。我们现在有两个相冲突的目标，一个是尽量减少违反，一个是加大间隔。引进C系数作为其中的折中参数。目标函数如下：  
$minimize_{\mathbf w,b,\zeta}\qquad \frac{1}{2}\mathbf w^T\cdot\mathbf w+C\sum_{i=1}^m\zeta^{(i)}$  
$subject\quad to\quad t^{(i)}(\mathbf w^T\cdot x^{(i)}+b)\ge 1-\zeta^{(i)}\qquad and\qquad \zeta^{(i)}\ge0 \quad for\quad i=1,2\cdots ,m$    
## 二次规划
上述两个目标函数都是线性约束条件下的二次曲线优化问题，被称作二次规划，一般表达式  
$ minimize_p \quad \frac{1}{2}\mathbf p^T\cdot \mathbf H\cdot\mathbf p +\mathbf f^T\cdot p $  
$ subject\quad to\quad\mathbf A\cdot p\le b $  
$ where\left{\begin{array}{ll}\mathbf p& \text{is an $n_p$-dimensional vector($n_p$=number of parameters)},\\
\mathbf H&\text{is an $n_p \times n_p$ matrix},\\
\mathbf f&\text{is an $n_p$-dimensional vector},\\
\mathbf A&\text{is an $n_c\times n_p$ matrix($n_c$=number of constraints)},\\
\mathbf b&\text{is an $n_c$-dimensional vector}. \end{array}\right.$

## 对偶问题
给定一个规划问题可以用表达不同但是紧密相关的另一个问题表示，称为对偶问题。通常情况下对偶问题只能求出原问题的下限，在某些情况下，对偶问题的解就是原问题的解，而SVM就包含了这些条件(目标函数是凸函数，约束条件是连续凸函数)  
SVM对偶的目标函数:  
$minimize_\alpha\quad \frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m\alpha^{(i)}\alpha^{(j)}t^{(i)}t^{(j)}x^{(i)T}\cdot x^{(j)} - \sum_{i=1}^m\alpha^{(i)} $  
$\text{subject to }\alpha^{(i)}\ge0\text{ for }i=1,2,\cdots,m  $  
一旦找到了合适的$\hat{\alpha}$,就能利用下列等式计算$\hat{\mathbf w},\hat{b}$  
$\hat{\mathbf w}=\sum_{i=1}^m\hat{\alpha}^{(i)}t^{(i)}x^{(i)}$  
$\hat{b}=\frac{1}{n_s}\sum_{i=1,\hat{\alpha}^{(i)}>0}^m(1-t^{(i)}(\hat{\mathbf w}^T\cdot x^{(i)}))$  
在样例数量比特征数量小时，求对偶问题比求解原问题更加快速。更重要的是，对偶问题可以使用核技巧。 
## 核技巧
二次多项式映射：  
$\phi(\mathbf x)=\phi\left(\left(\begin{array}{l} x_1\\x_2 \end{array}\right)\right)=\left(\begin{array}{l}x_1^2\\\sqrt2x_1x_2\\x_2^2\end{array}\right)$   
$\phi(\mathbf a)^T\cdot\phi(\mathbf b)=\left(\begin{array}{l}a_1^2\\\sqrt2a_1a_2\\a_2^2\end{array}\right)\left(\begin{array}{l}b_1^2\\\sqrt2b_1b_2\\b_2^2\end{array}\right)=a_1^2b_1^2+2a_1b_1a_2b_2+a_2^2b_2^2=(a_1b_1+a_2b_2)^2=(\mathbf a^T\cdot \mathbf b)^2$  
变换结果的点乘变成了原向量的点乘后平方！  
核技巧就是对所有样例应用变换，求对等问题时变换后的点乘用原样例点乘的平方替换，这样就可以不用实际对所有样例应用变换了，而且还将问题变成线性分隔问题，所以这使得计算变得效率。这种变换函数称为核函数，类似二次变换的$K(\mathbf a,\mathbf b)=(\mathbf a^T\cdot \mathbf b)^2$很多，下面我们列举一些常用的：  
$\text{Linear: }K(\mathbf a,\mathbf b)=\mathbf a^T\cdot \mathbf b $  
$\text{Polynomial: }K(\mathbf a,\mathbf b)=(\gamma \mathbf a^T\cdot\mathbf b+r)^d $  
$\text{Gaussian RBF: }K(\mathbf a,\mathbf b)=exp(-\gamma||\mathbf a-\mathbf b||^2)  $  
$\text{Sigmoid: }K(\mathbf a,\mathbf b)=tanh(\gamma\mathbf a^T\cdot \mathbf b+r) $  
应用核技巧求解对偶公式的时候，结果包含了$\phi(x^{(i)})$。实际上，$\hat{\mathbf w}$和$\phi(x^{(i)})$有相同的维数，这可能非常大，甚至无穷大，所以是无法计算的，那我们在不知道$\hat{\mathbf w}$怎么预测呢？  


一个好办法就是将它塞入新样例向量$\mathbf x^{(n)}$的判决函数，然后将得到输入向量点乘的一个表达式:  
$h_{\hat{\mathbf w},\hat b}(\phi(\mathbf x^{(n)})+\hat b=(\sum_{i=1}^m\hat\alpha^{(i)}t^{(i)}\phi(\mathbf x^{(i)}))^T\cdot\phi(\mathbf x^{(n)})+\hat b\\ \qquad=\sum_{i=1}^m\hat\alpha^{(i)}t^{(i)}(\phi(\mathbf x^{(i)})^T\cdot\phi(\mathbf x^{(n)}))+\hat b \\ \qquad=\sum_{i=1,\hat\alpha^{(i)}>0}^m\hat\alpha^{(i)}t^{(i)}K(\mathbf x^{(i)},\mathbf x^{(n)})+\hat b$   
注意只有支持向量的$\hat\alpha^{(i)}不为0$。因此每次加入新样例计算时，只调用支持向量的数据，而不是整个数据集，偏差项$\hat b$的计算也是如此：  
$\hat b=\frac{1}{n_s}\sum_{i=1,\hat\alpha^{(i)}>0}^m(1-t^{(i)}\hat{\mathbf w}^T\cdot\phi(\mathbf x^{(i)}))
\\\quad=\frac{1}{n_s}\sum_{i=1,\hat\alpha^{(i)}>0}^m(1-t^{(i)}(\sum_{j=1}^m\hat\alpha^{(j)}t^{(j)}\phi(\mathbf x^{(j)}))^T\cdot\phi(\mathbf x^{(i)}))\\
\quad=\frac{1}{n_s}\sum_{i=1,\hat\alpha^{(i)}>0}^m(1-t^{(i)}\sum_{j=1,\hat\alpha{(j)}>0}^m\hat\alpha^{(j)}t^{(j)}K(\mathbf x^{(i)},\mathbf x^{(j)}))$

# 在线SVMs
对于线性SVM，我们可以应用BGD方法最小化损失函数，但是这比QP方法慢很多。  
线性SVM分类的损失函数  
$J(\mathbf w,b)=\frac{1}{2}\mathbf w^T\cdot\mathbf w+C\sum_{i=1}^mmax(0,1-t^{(i)}(\mathbf w^T\cdot \mathbf x^{(i)}+b))$  
第一个和项使用小权值的$\mathbf w$向量，主导扩大分隔距离。第二个和项计算间隔违反。  
$max(0,1-t)$称为铰链损失函数hinge loss  
  
还有一些kernel SVM可以用于在线算法，但是在处理大量分线性数据时，应该考虑神经网络来替代它。